In [ ]:
#https://gist.github.com/dceoy/99d976a2c01e7f0ba1c813778f9db744

import io
import os
import pandas as pd
import gzip
import numpy as np
from google.cloud import storage
from depmap_omics_upload import tracker as track


def read_vcf(path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(path.split("/")[2])
    blob = bucket.blob("/".join(path.split("/")[3:]))
    if path.endswith(".gz"):
        data = io.BytesIO(blob.download_as_string())
        with gzip.open(data, 'r') as f:
            lines = [l.decode("utf-8") for l in f if not l.startswith(b'#')]
    else:
        data = blob.download_as_string().decode("utf-8") 
        f = data.split("\n")
        lines = [l + "\n" for l in f if not l.startswith('#')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'GT'],
        sep='\t'
    )

In [ ]:
def transformGB(row):
    if len(row) < 2:
        return np.nan
    else:
        split_gt = row.split(':')[1].split('|')
        return [int(i) for i in split_gt]

In [ ]:
def gb2str(row):
    if not isinstance(row["GB"], list):
        return "NA"
    else:
        strs = []
        for i in row["GB"]:
            integer = (row["REF"] * row["PERIOD"] + i) // row["PERIOD"]
            remainder = (row["REF"] * row["PERIOD"] + i) % row["PERIOD"]
            if remainder == 0:
                strs.append(str(int(integer)))
            else:
                strs.append(str(int(integer)) + "." + str(int(remainder)))
        return ', '.join(list(set(strs)))

In [ ]:
def generateSTRRow(paths_df, method, colname="str"):
    hg38_sites = pd.read_csv("/home/xiaomeng/bin/depmap_omics/data/str_hg38.bed", sep="\t", names=["CHROM","START","END","PERIOD","REF","ID"]).astype({'PERIOD': 'int32'})
    str_rows = []
#     mytracker = track.SampleTracker()
#     seq_table = mytracker.add_model_cols_to_seqtable(cols=["ModelID"])
#     mytracker.close_gumbo_client()
    for i, p in paths_df.iterrows():
        if pd.isna(p[colname]):
            print("no hipSTR vcf available for: " + i)
        else:
            df = read_vcf(p[colname])
            if method == "hipstr":
                df["GB"] = df.apply(lambda x: (transformGB(x["GT"])), axis=1)
                df = hg38_sites.merge(df[["ID", "GB"]], on='ID', how='left')
                df["STR"] = df.apply(lambda x: gb2str(x), axis=1)
            elif method == "gangstr":
                df = df.rename(columns={"POS": "START"})
                df = hg38_sites.merge(df[['CHROM', 'START', "ALT"]], on=['CHROM', 'START'], how='left')
                df["STR"] = df.apply(lambda x: altAllele2str(x), axis=1)
            # Maybe no need to convert to model IDs here yet?
            df["sample_id"] = i
            str_row = df.pivot(index='sample_id', columns='ID', values='STR')
            str_rows.append(str_row)
    return(pd.concat(str_rows))

In [ ]:
locations_14 = ['CSF1PO', 'D13S317', 'D16S539', 'D18S51', 'D21S11', 'D3S1358', 'D5S818', 'D7S820', 'D8S1179', 'FGA', 'PentaD', 'PentaE', 'TH01', 'TPOX']
locations_13 = ['CSF1PO', 'D13S317', 'D16S539', 'D18S51', 'D3S1358', 'D5S818', 'D7S820', 'D8S1179', 'FGA', 'PentaD', 'PentaE', 'TH01', 'TPOX']

In [ ]:
def computeTanabe(df1, idx1, df2, idx2, colnames=locations_13):
    # compute tanabe similarity between two STR profiles
    match = 0
    total = 0
    for col in colnames:
        # TODO: how to best handle NAs?
        a1 = set(df1.loc[idx1, col].split(", "))
        a2 = set(df2.loc[idx2, col].split(", "))
        if a1 != set(['NA']) & a2 != set(['NA']):
            match += len(set(a1) & set(a2))
            total += len(set(a1)) + len(set(a2))
    return(2 * match / total)

In [ ]:
def makeScoreMatrixDatabase(df_seqid, df_achid, colnames=locations_13):
    mytracker = track.SampleTracker()
    seq_table = mytracker.add_model_cols_to_seqtable(cols=["ModelID"])
    valid_achids = list(set(df_achid.index) - set([np.nan]))
    scoremat = pd.DataFrame(columns=valid_achids + ['ModelID'], index=(df_seqid.index))
    for i in df_seqid.index:
        scoremat.loc[i, "ModelID"] = seq_table.loc[i, "ModelID"]
        for j in valid_achids:
            scoremat.loc[i, j] = computeTanabe(df_seqid, i, df_achid, j)
    return scoremat

In [ ]:
fn1 = 'gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/fb75cad6-1ee7-41b9-b25d-d725a87067a2/hipstr/6d988cc2-8510-40f5-b77e-cbbe13b29ec4/call-run_hipstr/CDS-0b4jFH.vcf.gz'
fn2 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/fb75cad6-1ee7-41b9-b25d-d725a87067a2/hipstr/b364a608-44d1-4ac0-8abe-86bf7651d7e5/call-run_hipstr/CDS-00Nrci.vcf.gz"
fn3 = 'gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/fb75cad6-1ee7-41b9-b25d-d725a87067a2/hipstr/5eec577a-04f9-4a57-a056-c600e4b71fca/call-run_hipstr/CDS-0bV15m.vcf.gz'
fn4 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/fb75cad6-1ee7-41b9-b25d-d725a87067a2/hipstr/636ae126-8a85-4e19-b08c-589a28b2f5d2/call-run_hipstr/CDS-0e3PRe.vcf.gz"
fn5 = 'gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/fb75cad6-1ee7-41b9-b25d-d725a87067a2/hipstr/1423f686-fbfe-48db-b582-a1f23d56d7dc/call-run_hipstr/CDS-0ewUnk.vcf.gz'

In [ ]:
import dalmatian as dm

ws = dm.WorkspaceManager("broad-firecloud-ccle/DEV_DepMap_WGS_CN")
samples = ws.get_samples()

In [ ]:
samples = samples.iloc[0:50]

In [ ]:
samples

In [ ]:
path_mapping = pd.DataFrame(data=[fn1, fn2, fn3, fn4, fn5], columns=["str"], index=["CDS-0b4jFH", "CDS-00Nrci", "CDS-0bV15m", "CDS-0e3PRe", "CDS-0ewUnk"])

In [ ]:
wgs_str_hipstr = generateSTRRow(path_mapping, method="hipstr")

In [ ]:
wgs_str_hipstr

In [ ]:
from gsheets import Sheets

broad_internal_str_url = "https://docs.google.com/spreadsheets/d/134zxrQ77yMdDL4hLYybJJQN6pxLZIVLz-hORXIpid50/edit#gid=1107673601"
MY_ID = "~/.client_secret.json"
MYSTORAGE_ID = "~/.storage.json"
str_names = ['D3S1358','TH01', 'D21S11', 'D18S51', 'Penta E', 'D5S818', 'D13S317', 'D7S820',
             'D16S539', 'CSF1PO', 'Penta D', 'D8S1179', 'TPOX', 'FGA']

sheets = Sheets.from_files(MY_ID, MYSTORAGE_ID)
broad_str = sheets.get(broad_internal_str_url).sheets[0].to_frame(index_col=0)[str_names + ['Source']].rename(columns={"Penta E": "PentaE", "Penta D": "PentaD"})
broad_str = broad_str.fillna("NA")
broad_str = broad_str[~broad_str.index.isin(set([np.nan]))]

In [ ]:
minerva_str = sheets.get("https://docs.google.com/spreadsheets/d/1RsJS2e6zgyHwnt4bUyedYoyMbXMNbSmZfsr7SoSKn5U/edit?usp=sharing").sheets[0].to_frame(index_col=0)[str_names].rename(columns={"Penta E": "PentaE", "Penta D": "PentaD"})
minerva_str = minerva_str.fillna("NA")
minerva_str = minerva_str[~minerva_str.index.isin(set([np.nan]))]

In [ ]:
# drop dup rows, if from multiple source, keep the one from achilles
idx = broad_str.index.tolist()
dups = set([x for x in idx if idx.count(x) > 1])
broad_str = broad_str.reset_index()
todrop = []
for d in dups:
    dup_rows = broad_str[broad_str['Arxspan ID'] == d]
    if len(set(dup_rows.Source)) > 1 and "Achilles" in set(dup_rows.Source):
        todrop.extend(dup_rows[dup_rows['Source'] != "Achilles"].index.tolist())
broad_str = broad_str.drop(todrop)
broad_str = broad_str.drop(columns=["Source"])
broad_str = broad_str.set_index('Arxspan ID')

In [ ]:
broad_str

In [ ]:
score_mat = makeScoreMatrixDatabase(wgs_str, broad_str)

In [ ]:
score_mat_minerva = makeScoreMatrixDatabase(wgs_str, minerva_str)

In [ ]:
wgs_str.loc[["CDS-00Nrci"]][locations_13]

In [ ]:
broad_str.loc[["ACH-000839"]][locations_13]

In [ ]:
score_mat_minerva_t.loc["2869-1", "CDS-Zdl1W4"]

In [ ]:
score_mat_minerva_t = score_mat_minerva.T
score_mat_minerva_t = score_mat_minerva_t.drop(["ModelID"])
score_mat_minerva_t[score_mat_minerva_t["CDS-Zdl1W4"].astype(float) > 0.6]

In [ ]:
mytracker = track.SampleTracker()
seq_table = mytracker.add_model_cols_to_seqtable(cols=["ModelID", "PatientID"])

model_table = mytracker.read_model_table()

In [ ]:
scores = []
mismatch_scores = []
for i in score_mat.index:
    if score_mat.loc[i, "ModelID"] in set(score_mat.columns):
        patient_id = seq_table.loc[i, "PatientID"]
        same_patient = set(model_table[model_table.PatientID == patient_id].index)
        scores.append(score_mat.loc[i, score_mat.loc[i, "ModelID"]])
        mismatches = set(score_mat.columns) - set(["ModelID", score_mat.loc[i, "ModelID"]]) - same_patient
        mismatch_scores.extend(score_mat.loc[i, list(mismatches)].tolist())

In [ ]:
score_mat[score_mat.ModelID == "ACH-002835"]["ACH-002835"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize =(6, 4))
ax.hist(scores, bins = 20)
 
# Show plot
plt.show()

In [ ]:
# plot mismatch scores:

fig, ax = plt.subplots(figsize =(6, 4))
ax.hist(mismatch_scores, bins = 20)
 
# Show plot
plt.show()

In [ ]:
mismatch_scores.sort()

In [ ]:
mismatch_scores[-20:]

# GangSTR

In [ ]:
def altAllele2str(row):
    if pd.isna(row["ALT"]) or row["ALT"] == ".":
        return "NA"
    else:
        strs = []
        for i in row["ALT"].split(","):
            l = len(i)
            integer = l // row["PERIOD"]
            remainder = l % row["PERIOD"]
            if remainder == 0:
                strs.append(str(int(integer)))
            else:
                strs.append(str(int(integer)) + "." + str(int(remainder)))
        return ', '.join(list(set(strs)))

In [ ]:
gangstr_fn1 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/7b4fe301-f573-4859-bd05-98191c15a608/GangSTR/84e4fe9a-096e-4613-a0fd-462a1081a294/call-CallGangSTR/attempt-2/CDS-00Nrci.vcf"
gangstr_fn2 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/7b4fe301-f573-4859-bd05-98191c15a608/GangSTR/1bea68db-ec31-4e18-b2e0-9c76fce185ed/call-CallGangSTR/attempt-2/CDS-0b4jFH.vcf"
gangstr_fn3 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/7b4fe301-f573-4859-bd05-98191c15a608/GangSTR/643addc8-e6f0-45ae-812e-79fa599b3328/call-CallGangSTR/CDS-0bV15m.vcf"
gangstr_fn4 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/7b4fe301-f573-4859-bd05-98191c15a608/GangSTR/4d798622-7f63-41b2-8a05-5c06287556f9/call-CallGangSTR/CDS-0e3PRe.vcf"
gangstr_fn5 = "gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/7b4fe301-f573-4859-bd05-98191c15a608/GangSTR/d987a65c-0cd0-48d0-9e8d-a439de082dcc/call-CallGangSTR/attempt-2/CDS-0ewUnk.vcf"

In [ ]:
path_mapping_gangstr = pd.DataFrame(data=[gangstr_fn1, gangstr_fn2, gangstr_fn3, gangstr_fn4, gangstr_fn5], columns=["str"], index=["CDS-00Nrci", "CDS-0b4jFH", "CDS-0bV15m", "CDS-0e3PRe", "CDS-0ewUnk"])

In [ ]:
wgs_str_gangstr = generateSTRRow(path_mapping_gangstr, method="gangstr")

In [ ]:
wgs_str_gangstr.loc[["CDS-00Nrci"]]

In [ ]:
wgs_str_hipstr.loc[["CDS-00Nrci"]]

In [ ]:
df = read_vcf(gangstr_fn1)
df

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

CCLE_gene_cn = tc.get(name='internal-20q4-2540', version=47, file='CCLE_gene_cn')

In [ ]:
"ACH-001955" in CCLE_gene_cn.index

In [ ]:
wgs_str_gangstr_all = generateSTRRow(samples, method="gangstr", colname="GangSTR_vcf")

In [ ]:
wgs_str_gangstr_all

In [ ]:
score_mat_gangstr = makeScoreMatrixDatabase(wgs_str_gangstr_all, broad_str, colnames=locations_14)

In [ ]:
score_mat_gangstr

In [ ]:
scores_gangstr = []
mismatch_scores_gangstr = []
for i in score_mat_gangstr.index:
    if score_mat_gangstr.loc[i, "ModelID"] in set(score_mat_gangstr.columns):
        patient_id = seq_table.loc[i, "PatientID"]
        same_patient = set(model_table[model_table.PatientID == patient_id].index)
        scores_gangstr.append(score_mat_gangstr.loc[i, score_mat_gangstr.loc[i, "ModelID"]])
        mismatches = set(score_mat_gangstr.columns) - set(["ModelID", score_mat_gangstr.loc[i, "ModelID"]]) - same_patient
        mismatch_scores_gangstr.extend(score_mat_gangstr.loc[i, list(mismatches)].tolist())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize =(6, 4))
ax.hist(scores_gangstr, bins = 20)
 
# Show plot
plt.show()

In [ ]:
# plot mismatch scores:

fig, ax = plt.subplots(figsize =(6, 4))
ax.hist(mismatch_scores_gangstr, bins = 20)

# Show plot
plt.show()

In [ ]:
# check ACH-000600 and ACH-000658 against STR database

In [ ]:
ids = ["CDS-0DgVJy", "CDS-p5raBU", "CDS-2FC7DW"]

In [ ]:
import dalmatian as dm

wm = dm.WorkspaceManager("broad-firecloud-ccle/CCLE_SNP_QC-copy").disable_hound()

In [ ]:
mytracker = track.SampleTracker()
seq_table = mytracker.read_seq_table()

samples = seq_table.loc[ids]

In [ ]:
HG38BAMCOL = ["bam_filepath", "bai_filepath"]

bams = samples[HG38BAMCOL]
bams["id"] = bams.index

# Upload sample sheet
samples_df = pd.DataFrame(
    bams[HG38BAMCOL + ["id", "id"]].values,
    columns=["internal_bam_filepath", "internal_bai_filepath"] + ["sample_id", "participant_id"],
)

In [ ]:
samples_df = samples_df.set_index("sample_id")
wm.upload_samples(samples_df, add_participant_samples=True)

In [ ]:
fn1 = "gs://fc-secure-9c2c0017-9eb5-47e5-9f43-983097f1f7b6/submissions/5160df09-c8f6-47c1-a1b0-1016aad12f93/hipstr/09f55f83-7f97-4e61-9033-d34b950b9456/call-run_hipstr/CDS-0DgVJy.vcf.gz"
fn2 = "gs://fc-secure-9c2c0017-9eb5-47e5-9f43-983097f1f7b6/submissions/5160df09-c8f6-47c1-a1b0-1016aad12f93/hipstr/1be62168-30cf-491b-8227-c7f1876ac603/call-run_hipstr/CDS-p5raBU.vcf.gz"
fn3 = "gs://fc-secure-9c2c0017-9eb5-47e5-9f43-983097f1f7b6/submissions/5160df09-c8f6-47c1-a1b0-1016aad12f93/hipstr/86b096e1-53db-4665-8dfa-c731885d7312/call-run_hipstr/cacheCopy/CDS-2FC7DW.vcf.gz"

In [ ]:
path_mapping = pd.DataFrame(data=[fn1, fn2, fn3], columns=["str"], index=["CDS-0DgVJy", "CDS-p5raBU", "CDS-2FC7DW"])

In [ ]:
path_mapping = pd.DataFrame(data=[fn3], columns=["str"], index=["CDS-2FC7DW"])

In [ ]:
wgs_str_hipstr = generateSTRRow(path_mapping, method="hipstr")

In [ ]:
score_mat = makeScoreMatrixDatabase(wgs_str_hipstr, broad_str)

In [ ]:
score_mat.loc[["CDS-0DgVJy", "CDS-p5raBU", "CDS-2FC7DW"], ["ACH-001341", "ACH-000658"]]

In [ ]:
wgs_str_hipstr